# Training model

In [ ]:
import tensorflow as tf
import os
from keras.utils import img_to_array, load_img
import numpy as np
import cv2 as cv
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

trainPath = '/kaggle/input/fruits/fruits-360_dataset/fruits-360/Training'
testPath = '/kaggle/input/fruits/fruits-360_dataset/fruits-360/Test'

batchSize = 64      # Reduce value if you have less GPU

# # Read in example image and get shape
# img = load_img(trainPath + "/Quince/0_100.jpg")
# plt.imshow(img)
# plt.show()

# imgA = img_to_array(img)
# print(imgA.shape)        
# Build model
model = Sequential()
model.add(Conv2D(filters=128, kernel_size=3, activation="relu", input_shape=(100,100,3)))
model.add(MaxPooling2D())
model.add(Conv2D(filters=64, kernel_size=3, activation="relu"))
model.add(Conv2D(filters=32, kernel_size=3, activation="relu"))
model.add(MaxPooling2D())
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(5000, activation="relu"))
model.add(Dense(1000, activation="relu"))
model.add(Dense(131, activation="softmax"))

print(model.summary())


# Compile model
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# Load data
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.3, horizontal_flip=True, vertical_flip=True, zoom_range=0.3)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(trainPath, target_size=(100,100), batch_size=batchSize, color_mode="rgb", class_mode="categorical", shuffle=True)


test_generator = test_datagen.flow_from_directory(testPath, target_size=(100,100), batch_size=batchSize, color_mode="rgb", class_mode="categorical")

stepsPerEpoch = np.ceil(train_generator.samples / batchSize)
validationSteps = np.ceil(test_generator.samples / batchSize)

# Early stopping
stop_early = EarlyStopping(monitor="val_accuracy", patience=5)      # Stop fitting model if it doesn't improve by 5

history = model.fit(train_generator, steps_per_epoch=stepsPerEpoch, epochs=5, validation_data=test_generator, validation_steps=validationSteps, callbacks=[stop_early])

model.save("/kaggle/working/fruits360.h5")     # Add file path to save the model to

# Testing model

In [ ]:
!pip uninstall opencv-python-headless -y 
!pip install opencv-python --upgrade
import tensorflow as tf
import os
from keras.utils import img_to_array, load_img
import numpy as np
import cv2 as cv

# Load model
model = tf.keras.models.load_model("/kaggle/working/fruits360.h5")       # Put in model path here
print(model.summary())

# Load categories
source_folder = "/kaggle/input/fruits/fruits-360_dataset/fruits-360/Test"
categories = os.listdir(source_folder)
# categories = categories.sort()
print(categories)
print(len(categories))      # Should be 131

# Load and prepare image
def prepareImage(path):
    img = load_img(path, target_size=(100,100))
    imgArray = img_to_array(img)
    # print(imgArray.shape)
    imgArray = np.expand_dims(imgArray, axis=0)
    imgArray = imgArray / 255.
    return imgArray

testImgPath = "/kaggle/input/eggplant/eggplant.jpg"
imageForModel = prepareImage(testImgPath)

resultArray = model.predict(imageForModel, verbose=1)
answers = np.argmax(resultArray, axis=1)
print(categories[answers[0]])

# Show image with text below
img = cv.imread(testImgPath)
cv.putText(img, categories[answers[0]],(0,50), cv.FONT_HERSHEY_COMPLEX, 1, (209, 19, 77), 2)
cv.imshow('img', img)
cv.waitKey(0)




# Hyperparameter Tuning

In [5]:
!pip install tensorflow-gpu==2.10
from tpot import TPOTClassifier
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

digits = load_digits()
X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target,
                                                    train_size=0.75, test_size=0.25)

tpot = TPOTClassifier(generations=5, population_size=50, verbosity=2)
tpot.fit(X_train, y_train)
print(tpot.score(X_test, y_test))
tpot.export('tpot_digits_pipeline.py')

ERROR: Invalid requirement: '=='


Optimization Progress:   0%|          | 0/300 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.9777282114828584

Generation 2 - Current best internal CV score: 0.9777282114828584

Generation 3 - Current best internal CV score: 0.9777282114828584

Generation 4 - Current best internal CV score: 0.9799586947542338

Generation 5 - Current best internal CV score: 0.9807049428610768

Best pipeline: MLPClassifier(MaxAbsScaler(input_matrix), alpha=0.01, learning_rate_init=0.01)
0.9755555555555555


# Hyperparameter Tuning 2

In [7]:
import keras_tuner as kt
import tensorflow as tf
import os
from keras.utils import img_to_array, load_img
import numpy as np
import cv2 as cv
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

#class MyHyperModel(kt.HyperModel):
def build(hp):
    model = Sequential()
    model.add(Conv2D(filters=128, kernel_size=3, activation=hp.Choice('activation', ['relu', 'PReLU', 'elu']), input_shape=(100,100,3)))
    model.add(MaxPooling2D())
    model.add(Conv2D(filters=64, kernel_size=3, activation="relu"))
    model.add(Conv2D(filters=32, kernel_size=3, activation="relu"))
    model.add(MaxPooling2D())
    model.add(Dropout(rate=hp.Float('rate',min_value=0.3,max_value=0.8,step=0.1)))
    model.add(Flatten())
    model.add(Dense(units=hp.Int('units',3000,8000,500), activation=hp.Choice('activation_2', ['relu', 'PReLU', 'elu'])))
    model.add(Dense(units=hp.Int('units_2',500,1500,250), activation="relu"))
    model.add(Dense(131, activation="softmax"))
    #model.add(keras.layers.Dense(
     #   hp.Choice('units', [8, 16, 32]),
      #  activation='relu'))
   # model.add(keras.layers.Dense(1, activation='relu'))
    model.compile(loss="categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=["accuracy"])
    
    print(model.summary())
    return model

#def fit(hp, model):
    # Early stopping
#stop_early = EarlyStopping(monitor="val_accuracy", patience=5)      # Stop fitting model if it doesn't improve by 5

#history = model.fit(train_generator, steps_per_epoch=stepsPerEpoch, epochs=5, validation_data=test_generator, validation_steps=validationSteps, callbacks=[stop_early])


trainPath = '/kaggle/input/fruits/fruits-360_dataset/fruits-360/Training'
testPath = '/kaggle/input/fruits/fruits-360_dataset/fruits-360/Test'

batchSize = 64      # Reduce value if you have less GPU

# Load data
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.3, horizontal_flip=True, vertical_flip=True, zoom_range=0.3)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(trainPath, target_size=(100,100), batch_size=batchSize, color_mode="rgb", class_mode="categorical", shuffle=True)


test_generator = test_datagen.flow_from_directory(testPath, target_size=(100,100), batch_size=batchSize, color_mode="rgb", class_mode="categorical")
#x = MyHyperMode(kt.HyperModel)
#build(kt.HyperParameters())
#tuner = kt.Hyperband(hypermodel = build, objective="val_accuracy", max_epochs=10, hyperband_iterations=10,seed=1,max_retries_per_trial=0, max_consecutive_failed_trials=3)
tuner = kt.RandomSearch(hypermodel = build, objective="val_accuracy", max_trials=10,executions_per_trial=1,overwrite=False)
#tuner.search_space_summary()
tuner.search(train_generator,epochs=2,validation_data=test_generator)
tuner.results_summary()

Trial 10 Complete [00h 11m 44s]
val_accuracy: 0.9436706900596619

Best val_accuracy So Far: 0.9436706900596619
Total elapsed time: 01h 27m 25s
Results summary
Results in ./untitled_project
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 09 summary
Hyperparameters:
activation: relu
rate: 0.4
units: 4000
activation_2: relu
units_2: 1500
learning_rate: 0.1
Score: 0.9436706900596619

Trial 07 summary
Hyperparameters:
activation: PReLU
rate: 0.3
units: 3500
activation_2: PReLU
units_2: 1250
learning_rate: 0.01
Score: 0.9378526210784912

Trial 05 summary
Hyperparameters:
activation: relu
rate: 0.4
units: 6000
activation_2: relu
units_2: 1500
learning_rate: 0.1
Score: 0.9287288188934326

Trial 03 summary
Hyperparameters:
activation: elu
rate: 0.4
units: 8000
activation_2: relu
units_2: 1250
learning_rate: 0.001
Score: 0.9250705242156982

Trial 08 summary
Hyperparameters:
activation: PReLU
rate: 0.6000000000000001
units: 5000
activation_2: relu
units_2: 500
learni